# LLM Function Calling - Vanilla Implementation

Learn how to implement **LLM function calling from scratch** without frameworks.

**What you'll learn:**
- Define tool schemas for function calling
- Use llama.cpp for local inference
- Parse and execute LLM-generated function calls

**Use Case:** Natural language → Git commands

## Setup

Install dependencies and the model will auto-download from HuggingFace (3GB).

In [ ]:
# Install llama-cpp-python
!uv add llama-cpp-python

In [1]:
import json
import subprocess
from llama_cpp import Llama

## Step 1: Define Tool Schemas

Tools are defined in JSON schema format. Each tool has:
- **name**: Function identifier
- **description**: What the function does (helps LLM choose correct tool)
- **parameters**: Input arguments with types

In [2]:
# All 18 Git commands as tool schemas
tools = [
    {
        "name": "git_config_username",
        "description": "Configure Git global username",
        "parameters": {
            "type": "object",
            "properties": {
                "username": {"type": "string", "description": "Git username"}
            },
            "required": ["username"]
        }
    },
    {
        "name": "git_config_email",
        "description": "Configure Git global email",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {"type": "string", "description": "Git email"}
            },
            "required": ["email"]
        }
    },
    {
        "name": "git_commit",
        "description": "Commit with a message",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {"type": "string", "description": "Commit message"}
            },
            "required": ["message"]
        }
    },
    {
        "name": "git_checkout",
        "description": "Checkout to a branch",
        "parameters": {
            "type": "object",
            "properties": {
                "branch_name": {"type": "string", "description": "Branch name"}
            },
            "required": ["branch_name"]
        }
    },
    {
        "name": "git_create_branch",
        "description": "Create a new branch",
        "parameters": {
            "type": "object",
            "properties": {
                "branch_name": {"type": "string", "description": "New branch name"}
            },
            "required": ["branch_name"]
        }
    },
    {
        "name": "git_delete_branch",
        "description": "Delete a branch",
        "parameters": {
            "type": "object",
            "properties": {
                "branch_name": {"type": "string", "description": "Branch to delete"}
            },
            "required": ["branch_name"]
        }
    },
    {
        "name": "git_rename_branch",
        "description": "Change the name of current branch/Rename current branch",
        "parameters": {
            "type": "object",
            "properties": {
                "old_name": {"type": "string", "description": "Current branch name"},
                "new_name": {"type": "string", "description": "New branch name"}
            },
            "required": ["old_name", "new_name"]
        }
    },
    {
        "name": "git_status",
        "description": "Show git status",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "git_reset_last_commit",
        "description": "Remove the last commit (soft/mixed/hard)",
        "parameters": {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["soft", "mixed", "hard"],
                    "description": "Reset mode"
                }
            },
            "required": ["mode"]
        }
    },
    {
        "name": "git_add_remote",
        "description": "Add a new remote",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Remote name"},
                "url": {"type": "string", "description": "Remote URL"}
            },
            "required": ["name", "url"]
        }
    },
    {
        "name": "git_remove_remote",
        "description": "Remove a remote",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Remote name"}
            },
            "required": ["name"]
        }
    },
    {
        "name": "git_list_remotes",
        "description": "List all remotes",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "git_add",
        "description": "Add files to staging",
        "parameters": {
            "type": "object",
            "properties": {
                "files": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Files to add"
                }
            },
            "required": ["files"]
        }
    },
    {
        "name": "git_unstage",
        "description": "Remove files from staging",
        "parameters": {
            "type": "object",
            "properties": {
                "files": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Files to unstage"
                }
            },
            "required": ["files"]
        }
    },
    {
        "name": "git_pull",
        "description": "Pull changes from remote",
        "parameters": {
            "type": "object",
            "properties": {
                "remote": {"type": "string", "description": "Remote name", "default": "origin"},
                "branch": {"type": "string", "description": "Branch name", "default": "main"}
            },
            "required": []
        }
    },
    {
        "name": "git_push",
        "description": "Push changes to remote",
        "parameters": {
            "type": "object",
            "properties": {
                "remote": {"type": "string", "description": "Remote name", "default": "origin"},
                "branch": {"type": "string", "description": "Branch name", "default": "main"}
            },
            "required": []
        }
    },
    {
        "name": "git_init",
        "description": "Initialize a new git repo",
        "parameters": {
            "type": "object",
            "properties": {
                "directory": {"type": "string", "description": "Directory path", "default": "."}
            },
            "required": []
        }
    },
    {
        "name": "git_clone",
        "description": "Clone a git repo",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {"type": "string", "description": "Repository URL"},
                "directory": {"type": "string", "description": "Target directory"}
            },
            "required": ["url"]
        }
    }
]

print(f"✓ Defined {len(tools)} Git commands as tool schemas")

✓ Defined 18 Git commands as tool schemas


In [12]:
# Load the GGUF model directly from HuggingFace
# Model auto-downloads on first run (~3GB)
model = Llama.from_pretrained(
    repo_id="afmjoaa/ngit-xLAM-lora-100-16bit",
    filename="unsloth.F16.gguf",
    n_ctx=2048,      # Context window
    n_threads=4,     # CPU threads
    verbose=False
)

print("✓ Model loaded successfully")

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

✓ Model loaded successfully


In [ ]:
# Load the GGUF model
model = Llama(
    model_path="./unsloth.F16.gguf",  # Path to downloaded model
    n_ctx=2048,                       # Context window
    n_threads=4,                      # CPU threads
    verbose=False
)

print("✓ Model loaded successfully")

## Step 3: Translation Function

Convert natural language to function calls.

**How it works:**
1. Format user message + tools into a prompt
2. LLM generates JSON with function name + arguments
3. Parse JSON response

In [13]:
def translate(user_message):
    """
    Translate natural language to function calls.
    
    Args:
        user_message: Natural language command (e.g., "Add all files")
    
    Returns:
        list: Parsed function calls with name and arguments
    """
    # Create prompt with tools and user message
    # The xLAM model expects this specific format
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a function calling AI model. Given a user query and available tools, output the function call(s) in JSON format.

Available tools:
{json.dumps(tools, indent=2)}

<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    # Generate response from model
    response = model(
        prompt,
        max_tokens=256,      # Limit response length
        temperature=0.0,     # Deterministic (no randomness)
        stop=["<|eot_id|>"]  # Stop at end of turn
    )
    
    # Extract generated text
    raw_output = response['choices'][0]['text'].strip()
    
    # Parse JSON (model outputs array of function calls)
    try:
        function_calls = json.loads(raw_output)
        return function_calls
    except json.JSONDecodeError:
        print(f"Error parsing JSON: {raw_output}")
        return None

## Step 4: Command Mapper

Convert function calls to actual shell commands.

In [14]:
# Map function names to shell command builders
COMMAND_MAP = {
    "git_config_username": lambda username: ["git", "config", "--global", "user.name", username],
    "git_config_email": lambda email: ["git", "config", "--global", "user.email", email],
    "git_commit": lambda message: ["git", "commit", "-m", message],
    "git_checkout": lambda branch_name: ["git", "checkout", branch_name],
    "git_create_branch": lambda branch_name: ["git", "branch", branch_name],
    "git_delete_branch": lambda branch_name: ["git", "branch", "-d", branch_name],
    "git_rename_branch": lambda old_name, new_name: ["git", "branch", "-m", old_name, new_name],
    "git_status": lambda: ["git", "status"],
    "git_reset_last_commit": lambda mode: ["git", "reset", f"--{mode}", "HEAD~1"],
    "git_add_remote": lambda name, url: ["git", "remote", "add", name, url],
    "git_remove_remote": lambda name: ["git", "remote", "remove", name],
    "git_list_remotes": lambda: ["git", "remote", "-v"],
    "git_add": lambda files: ["git", "add"] + files,
    "git_unstage": lambda files: ["git", "reset", "HEAD"] + files,
    "git_pull": lambda remote="origin", branch="main": ["git", "pull", remote, branch],
    "git_push": lambda remote="origin", branch="main": ["git", "push", remote, branch],
    "git_init": lambda directory=".": ["git", "init", directory],
    "git_clone": lambda url, directory=None: ["git", "clone", url] + ([directory] if directory else [])
}

def get_shell_command(function_call):
    """
    Convert function call to shell command.
    
    Args:
        function_call: Dict with 'name' and 'arguments'
    
    Returns:
        str: Shell-ready command
    """
    func_name = function_call['name']
    args = function_call['arguments']
    
    # Get command builder function
    builder = COMMAND_MAP.get(func_name)
    if not builder:
        return None
    
    # Build command tokens
    cmd_tokens = builder(**args)
    
    # Join into shell command (with proper quoting)
    return " ".join(f'"{token}"' if " " in token else token for token in cmd_tokens)

def execute_command(command):
    """
    Execute shell command.
    
    Args:
        command: Shell command string
    """
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ Success: {result.stdout}")
    else:
        print(f"✗ Error: {result.stderr}")
    return result

## Step 5: Complete Example

End-to-end workflow: Natural language → Translation → Review → Execution

In [15]:
def process_natural_language_command(user_input, auto_execute=False):
    print(f"\n User: {user_input}\n")

    # Step 1: Translate to function calls
    print(" Translating...")
    function_calls = translate(user_input)

    if not function_calls:
        print("❌ Translation failed")
        return

    # Step 2: Show translation result
    print("\n Translation Result:")
    print(json.dumps(function_calls, indent=2))

    # Step 3: Convert to shell command
    for fc in function_calls:
        shell_cmd = get_shell_command(fc)
        print(f"\n Shell Command: {shell_cmd}")

        # Step 4: Execute (with confirmation)
        if auto_execute:
            execute_command(shell_cmd)
        else:
            confirm = input("\n Execute this command? (y/n): ")
            if confirm.lower() == 'y':
                execute_command(shell_cmd)
            else:
                print(" Skipped")

### Example 2: Rename branch

In [16]:
process_natural_language_command("Change the develop branch name to development")


 User: Change the develop branch name to development

 Translating...

 Translation Result:
[
  {
    "name": "git_rename_branch",
    "arguments": {
      "old_name": "develop",
      "new_name": "development"
    }
  }
]

 Shell Command: git branch -m develop development
✗ Error: fatal: not a git repository (or any of the parent directories): .git



### Example 3: Commit changes

In [10]:
process_natural_language_command("Create a commit with message 'Initial commit'")


📝 User: Create a commit with message 'Initial commit'

🔄 Translating...

📋 Translation Result:
[
  {
    "name": "git_commit",
    "arguments": {
      "message": "Initial commit"
    }
  }
]

💻 Shell Command: git commit -m "Initial commit"
✗ Error: fatal: not a git repository (or any of the parent directories): .git

